In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import json

In [2]:
import os

# Define the data folder path
data_folder = 'data/question_1a/'

# Get all JSON files in the folder
json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]

# Loop through all JSON files and load them into separate variables
for filename in json_files:
    filepath = os.path.join(data_folder, filename)
    # Use filename without extension as variable name
    var_name = filename.replace('.json', '')
    
    with open(filepath, 'r') as f:
        # Create a variable with the same name as the file
        globals()[var_name] = json.load(f)
    
    print(f"Loaded: {filename} -> {var_name}")

# Now you have separate dictionaries:
# appliance_params
# bus_params
# consumer_params
# DER_production
# usage_preference

print(f"\nLoaded {len(json_files)} files as separate dictionaries:")
for filename in json_files:
    var_name = filename.replace('.json', '')
    print(f"  - {var_name}")

Loaded: appliance_params.json -> appliance_params
Loaded: bus_params.json -> bus_params
Loaded: consumer_params.json -> consumer_params
Loaded: DER_production.json -> DER_production
Loaded: usage_preference.json -> usage_preference

Loaded 5 files as separate dictionaries:
  - appliance_params
  - bus_params
  - consumer_params
  - DER_production
  - usage_preference


In [7]:
# Extract key parameters from the loaded data
print("=== EXTRACTING PARAMETERS ===")

# Bus parameters
bus_data = bus_params[0]
import_tariff = bus_data['import_tariff_DKK/kWh']
export_tariff = bus_data['export_tariff_DKK/kWh']
max_import = bus_data['max_import_kW']
max_export = bus_data['max_export_kW']
electricity_prices = bus_data['energy_price_DKK_per_kWh']  # 24-hour prices

# Time horizon
T = len(electricity_prices)  # 24 hours

print(f"Import tariff: {import_tariff} DKK/kWh")
print(f"Export tariff: {export_tariff} DKK/kWh")
print(f"Max import/export: {max_import}/{max_export} kW")

# PV parameters
pv_data = appliance_params['DER'][0]
pv_max_power = pv_data['max_power_kW']
pv_profile = DER_production[0]['hourly_profile_ratio']
pv_prod_hourly = [pv_max_power * ratio for ratio in pv_profile]

print(f"PV max power: {pv_max_power} kW")

# Load parameters
load_data = appliance_params['load'][0]
load_max_hourly = load_data['max_load_kWh_per_hour']
min_daily_consumption = usage_preference[0]['load_preferences'][0]['min_total_energy_per_day_hour_equivalent']

print(f"Load max hourly: {load_max_hourly} kWh/hour")
print(f"Min daily consumption: {min_daily_consumption} kWh")

print("\n=== ENERGY PRICES BY HOUR ===")
for t in range(T):
    print(f"Hour {t:2d}: {electricity_prices[t]:5.2f} DKK/kWh, PV production: {pv_prod_hourly[t]:4.2f} kW")

=== EXTRACTING PARAMETERS ===
Import tariff: 5 DKK/kWh
Export tariff: 4 DKK/kWh
Max import/export: 1000/500 kW
PV max power: 3.0 kW
Load max hourly: 3.0 kWh/hour
Min daily consumption: 8 kWh

=== ENERGY PRICES BY HOUR ===
Hour  0:  1.10 DKK/kWh, PV production: 0.00 kW
Hour  1:  1.05 DKK/kWh, PV production: 0.00 kW
Hour  2:  1.00 DKK/kWh, PV production: 0.00 kW
Hour  3:  0.90 DKK/kWh, PV production: 0.00 kW
Hour  4:  0.85 DKK/kWh, PV production: 0.00 kW
Hour  5:  1.01 DKK/kWh, PV production: 0.15 kW
Hour  6:  1.05 DKK/kWh, PV production: 0.42 kW
Hour  7:  1.20 DKK/kWh, PV production: 0.63 kW
Hour  8:  1.40 DKK/kWh, PV production: 0.45 kW
Hour  9:  1.60 DKK/kWh, PV production: 0.36 kW
Hour 10:  1.50 DKK/kWh, PV production: 0.63 kW
Hour 11:  1.10 DKK/kWh, PV production: 0.75 kW
Hour 12:  1.05 DKK/kWh, PV production: 2.55 kW
Hour 13:  1.00 DKK/kWh, PV production: 2.25 kW
Hour 14:  0.95 DKK/kWh, PV production: 1.65 kW
Hour 15:  1.00 DKK/kWh, PV production: 1.29 kW
Hour 16:  1.20 DKK/kWh, PV

In [73]:
#Reitterating parameters for clarity
Times = range(T) # Time horizon (24 hours)
PV_t = pv_prod_hourly # PV production per hour (kW) (No curtailment)
p_t = electricity_prices # Electricity prices per hour (DKK/kWh)
tau_imp = import_tariff # Import tariff (DKK/kWh)
tau_exp = export_tariff # Export tariff (DKK/kWh)
E_min = min_daily_consumption # Minimum daily consumption (kWh)


In [74]:
model = gp.Model("Energy_Optimization")

# Decision variables:
L_t = model.addVars(Times, lb=0, name="L_t")  # Load consumption (kW)
C_t = model.addVars(Times, lb=0, name="C_t")  # Energy curtailed from PV (kW)
G_imp_t = model.addVars(Times, lb=0, ub=max_import, name="G_imp_t")  # Grid import (kW)
G_exp_t = model.addVars(Times, lb=0, ub=max_export, name="G_exp_t")  # Grid export (kW)


In [75]:
#Objective function
model.setObjective(
    gp.quicksum(p_t[t] * (G_imp_t[t] - G_exp_t[t]) + 
                tau_imp * G_imp_t[t] + tau_exp * G_exp_t[t] for t in Times),
    GRB.MINIMIZE
)

In [76]:
#add constraints

Curtailment_constraint = [
	model.addLConstr(C_t[t] <= PV_t[t], name=f"Curtailment")
	for t in Times
]

Load_coverage_constraint = [
	model.addLConstr(gp.quicksum(L_t[t] for t in Times) >= E_min, name=f"Total_Load_Coverage")
	for t in Times
]
# Power balance constraint
power_balance_constraint = [
	model.addLConstr(G_imp_t[t] - G_exp_t[t] == L_t[t] - PV_t[t] + C_t[t], name="power_balance")
	for t in Times
]

In [77]:
#solve optimization problem
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS Creator Edition, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 72 rows, 96 columns and 696 nonzeros
Model fingerprint: 0x375edccd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [2e-01, 8e+00]
Presolve removed 72 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [78]:
#print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Total cost: {model.objVal:.2f} DKK")
    print("Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)")
    for t in Times:
        print(f"{t:4d} | {L_t[t].X:9.2f} | {C_t[t].X:15.2f} | {G_imp_t[t].X:15.2f} | {G_exp_t[t].X:15.2f} | {PV_t[t]:18.2f} | {p_t[t]:25.2f}")


Optimal solution found:
Total cost: 0.00 DKK
Hour | Load (kW) | Curtailment (kW) | Grid Import (kW) | Grid Export (kW) | PV Production (kW) | Electricity Price (DKK/kWh)
   0 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.10
   1 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.05
   2 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      1.00
   3 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.90
   4 |      0.00 |            0.00 |            0.00 |            0.00 |               0.00 |                      0.85
   5 |      0.15 |            0.00 |            0.00 |            0.00 |               0.15 |                      1.01
   6 |      0.42 |            0.00 |            0.00 |            0.00 |               0.42 |                

In [79]:
#sum L_t
print(f"\nTotal Load Consumption: {sum(L_t[t].X for t in Times):.2f} kWh")


Total Load Consumption: 13.47 kWh
